### LangChain Reasoning and Synthesis Demonstration
---
This notebook demonstrates the use of LangChain to perform reasoning tasks using local models via Ollama.
We will break down a complex question into logical steps, retrieve relevant information for each step, and synthesize a final answer.

The question we're going to answer is "Who was the U.S. president during the moon landing, and what was his policy on space exploration?"

Let's start by installing any packages we need 

In [1]:
!pip install wikipedia
!pip install langchain
!pip install langchain_community
!pip install langchain_ollama

Import any modules required

In [2]:
# The `try` block ensures any imports are loaded successfully, and if there is an error during import, it will be caught and printed. This is useful for debugging and ensuring that the necessary libraries are available before running the main code.

try:
    # Imports
    from langchain_ollama import OllamaLLM
    from langchain.prompts import PromptTemplate
    from langchain_core.runnables import RunnableMap
    import re
except ImportError as e:
    print(f"ImportError occurred: {e}")
except Exception as e:
    print(f"Unexpected error during import: {e}")

### Load Local Reasoning and Synthesis Models
Here, we initialize two models using Ollama:
- **Reasoning Model (`phi4-mini`)**: Used to break down questions into logical steps.
- **Synthesis Model (`Gemma3:1b`)**: Used to synthesize a final answer based on retrieved information.
The `try` block ensures that the models are loaded successfully, and if there is an error (e.g., Ollama is not running), it provides helpful debugging information.

In [3]:
try:
    reasoning_llm = OllamaLLM(model="phi4-mini")
    synthesis_llm = OllamaLLM(model="Gemma3:1b")
except Exception as e:
    print("❌ Failed to connect to Ollama or load model 'phi4-mini'.")
    print("💡 Make sure Ollama is running and the model is available:")
    print("    ollama run phi4-mini")
    print(f"Error details: {e}")
    exit(1)

### Define Reasoning Prompt
We define a prompt template to guide the reasoning model. The template instructs the model to break down a given question into logical steps.
The `PromptTemplate.from_template` method allows us to create a reusable template with placeholders (e.g., `{question}`) that can be dynamically filled with input data.

In [4]:
reasoning_prompt = PromptTemplate.from_template("""
You are a reasoning assistant. Break the following question into logical steps. Use the provided information to answer the question:

Question: {question}

Step-by-step breakdown:
""")

### Chain Prompt and Model
We use LangChain's Expression Language to chain the reasoning prompt with the reasoning model. This creates a pipeline where the input question is first processed by the prompt and then passed to the reasoning model for step-by-step breakdown.

In [5]:
step_chain = reasoning_prompt | reasoning_llm

### Invoke Reasoning Chain
We provide a question to the reasoning chain and invoke it to generate a step-by-step breakdown of the question.
The `invoke` method processes the input through the chain and returns the model's response.

In [6]:
question = "Who was the U.S. president during the moon landing, and what was his policy on space exploration?"
response = step_chain.invoke({"question": question})
print("💡 Reasoning steps from the model:\n")
print(response)

💡 Reasoning steps from the model:

To find out who was the President of the United States (U.S.) at the time of Neil Armstrong's first step onto the Moon in 1969 as well as their policies regarding space exploration:

1. Identify when Apollo 11 landed: The moon landing occurred during NASA’s Apollo program, specifically on July 20th, 1969.
2. Determine who was President then: Research indicates that Richard Nixon served two terms and his second term overlapped with the time of this significant event in U.S space exploration history.

Based on these steps:

- Richard Nixon (President) 

As for Nixon's policy regarding Space Exploration:
3. Analyze major policies announced or enacted by John F. Kennedy prior to Nixon taking office: In 1961, President Johnson declared an ambitious goal that NASA was tasked with sending a manned mission to the moon and returning him safely before December of any year.
4. Check what followed after these goals were set in place during Jimmy Carter's presiden

### Call the Wikidpedia API
This code uses LangChain tools to interact with Wikipedia. It breaks a question into reasoning steps, retrieves facts from Wikipedia for each step, and displays the results.

### Key Steps

1. **Initialize Tools**: Set up `WikipediaAPIWrapper` and `WikipediaQueryRun` for Wikipedia lookups.
2. **Process Question**: Use `step_chain.invoke()` to break the question into steps.
3. **Retrieve Facts**: Look up each step using the Wikipedia tool, handle errors, and collect results.
4. **Display Results**: Combine retrieved facts into a single output and print them.

### Parse and Retrieve Information
In this step, we extract reasoning steps from the output of step_chain.invoke() using a regular expression (re.findall). The regular expression is designed to match the lines containing the numbered steps of reasoning.

For each extracted step, we use the wiki_tool.run() method to perform a Wikipedia lookup. If any errors occur during the lookup, they are caught and handled gracefully by capturing the exception. The result of each lookup (whether it’s a fact from Wikipedia or an error message) is stored in a list called facts.

Once all the steps have been processed, the individual facts are combined into a single string (combined_facts) and printed for further review or processing.

In [7]:
# --- Correct setup for Wikipedia Tool ---
from langchain.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wiki_api = WikipediaAPIWrapper(top_k_results=1, lang="en")
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)

# --- Use Wikipedia for each reasoning step ---

steps_text = step_chain.invoke({"question": question})

print("\n🧠 Reasoning steps:\n")
print(steps_text)

# Extract lines that look like steps
step_lines = re.findall(r"\d+\.\s+(.*)", steps_text)
facts = []

for step in step_lines:
    print(f"🔍 Looking up: {step}")
    try:
        result = wiki_tool.run(step)
    except Exception as e:
        result = f"Error retrieving from Wikipedia: {e}"
    facts.append(f"- {step.strip()}: {result}")

combined_facts = "\n".join(facts)

print("\n📚 Retrieved facts from Wikipedia:\n")
print(combined_facts)



🧠 Reasoning steps:

Sure! Let's break down this problem step by step.

1. Identify when NASA's Apollo 11 mission successfully landed humans on the Moon.
   - The successful manned lunar landing occurred in July 1969 (July 20th specifically).

2. Determine which U.S. president was serving at that time:
   - Richard Nixon served as President of the United States from January 20, 1969 to August 9, 1974.

3. Investigate Dick Cheney's involvement during this period in terms of his role and activities.
   - As George H.W. Bush had not yet been elected (he would take office later), Richard Nixon was clearly serving as the president at that time since he held presidential duties until leaving for retirement before Reagan took over directly.

4. Look into Dick Cheney's background:
   - In this context, it's important to note that there is no direct evidence linking George H.W. Bush and space exploration during his early career stage prior to becoming Vice President in 1976 (serving as VP under

### Define Synthesis Prompt
We define a prompt template to guide the synthesis model. The template combines the question and retrieved facts to generate a complete answer.
The `PromptTemplate.from_template` method is used to create a structured template with placeholders for the question and facts.

In [8]:
synthesis_prompt = PromptTemplate.from_template("""
Based on the following question and information, write a complete answer.

Question: {question}

Information:
{facts}

Answer:
""")
synthesis_chain = synthesis_prompt | synthesis_llm

### Generate Final Answer
We invoke the synthesis chain to generate the final answer based on the question and the retrieved facts.
The `invoke` method processes the input through the synthesis chain and returns the model's response, which is the final synthesized answer.

In [9]:
final_answer = synthesis_chain.invoke({
    "question": question,
    "facts": combined_facts
})
print("\n🧠 Final answer from synthesis model:\n")
print(final_answer)


🧠 Final answer from synthesis model:

Here’s a breakdown of the information provided and the answer to your question:

**Answer:**

Richard Nixon’s policy on space exploration was largely focused on **long-term scientific research and technological advancement, rather than immediate militaristic expansion.** He prioritized funding for NASA’s aeronautical programs, including the development of the Saturn V rocket, but he also emphasized the importance of scientific discovery and technological innovation, not primarily for military applications.

**Here’s a summary of his approach:**

*   **Focus on Science:** He saw space exploration as a crucial area of scientific research and technological development.
*   **Funding Priorities:** He allocated significant funding to NASA's aeronautical programs – including the development of rockets and space vehicles.
*   **“Silent Majority” Strategy:** His campaign emphasized appealing to the "silent majority," which was a broader appeal encompassin